::: {.callout-caution title="Citation"}
@DBLP:journals/sensors/SalmaneVKMDCSV23

[Paper link](https://www.mdpi.com/1424-8220/23/6/3223)
:::

## Related work

- A 3D bounding box is represented by seven parameters [@DBLP:conf/cvpr/QiLWSG18]: $(x,y,z,h,w,l,\theta)$ These parameters consist of the object's central coordinates $(x,y,z)$, its dimensions (height, width, and length), and its orientation angle $\theta$. 
- @DBLP:conf/cvpr/Chen0SJ20  introduced a one-stage stereo-based 3D detection system that simultaneously estimates depth and identifies 3D objects using an end-to-end learning approach. They assert that their method surpasses earlier stereo-based 3D detectors and even rivals certain LiDAR-based techniques on the KITTI 3D object detection leaderboard.
- @DBLP:conf/iros/LiKW20, addressed the issue of significant variability in depth estimation accuracy when using a video sensor. They introduce CG-Stereo, a confidence-guided stereo 3D object detection system. This pipeline employs distinct decoders for foreground and background pixels during the depth estimation process and utilizes confidence estimation from the depth estimation network as a form of soft attention in the 3D object detection. The authors contend that their method surpasses all leading stereo-based 3D detectors on the KITTI benchmark.
- Another compelling approach found in scholarly work involves integrating LiDAR with a stereo camera. Such techniques leverage LiDAR's ability to enhance the visual data captured by the camera by introducing concepts of dimension and distance pertaining to various objects within the environment. Specifically, the method outlined by @DBLP:conf/cvpr/WangCGHCW19 capitalizes on the capability to recreate a 3D setting from stereo camera images. This allows for the derivation of a depth map from the stereo camera data, which is then augmented with measurements from the LiDAR sensor, such as height, width, length, and orientation angle.

- SLS fusion, sparse LiDAR and stereo fusion network [@DBLP:journals/corr/abs-2103-03977]  which is based on DeepLiDAR [@DBLP:conf/cvpr/QiuCZZLZP19] and the pseudo-LiDAR pipeline [@DBLP:conf/cvpr/WangCGHCW19] used information coming from four beam LiDAR and a stereo camera. Fusion improved the depth estimation. resulting in better dense depth map, inturn improving the performance. 
- 3D object detection methods are classified according to the type of input data: camera-based, LiDAR-based, and fusion based 3D object detection.
- For 3D object detection, LiDAR-based methods are usually classified into four categories: view-based, voxel-based, point-based, and hybrid point-voxel-based detection.
- it is difficult to distinguish whether it is a car or a bush based on point cloud data alone, while this can be handled more easily by looking at the image data. This is why methods based on data fusion have been developed exploiting the advantages of both sensors
- There are three main fusion methods: early fusion, where the raw data are fused at the data level or feature level to form a tensor data of numerous channels; late fusion, where the fusion takes place at the decision level; and deep fusion, where fusion is carefully constructed to combine the advantages of both early and late fusion systems.
- Frustum-PointNet [@DBLP:conf/cvpr/QiLWSG18] is composed of three phases: 3D frustum proposal , 3D instance segmentation, and 3D bounding box estimation. 
    - The first phase of this procedure is to produce 2D region proposals. By extruding the matching 2D region proposal under a 3D projection, a 3D frustum proposal is generated.
    - The instance segmentation stage feeds the frustum proposal point cloud to the PointNet segmentation network [@DBLP:conf/cvpr/QiSMG17], which classifies each point and determines if it is linked with the discovered item.
    - In the last stage, all positively classified points are loaded into a new PointNet that estimates 3D bounding box parameters.
- @DBLP:conf/cvpr/ChenMWLX17 introduced MV3D, where the LiDAR point cloud is projected onto  both a 2D top view and a 2D front view, from which feature maps are extracted using two  separate CNN. The LiDAR top-view feature map is passed to an RPN (Region Proposal  Network) to output proposal 3D bounding boxes. Each 3D proposal is projected onto the  feature maps of all three views and a fixed-size feature vector is extracted for each view  using pooling. The three feature vectors are then fused in a region-based fusion network,  which finally outputs class scores and regresses 3D bounding box residuals.

    

## Analysis of the Role of Each Sensor in the 3D Object Detection Task

SLS–Fusion is a fusion method for LiDAR and stereo cameras based on a deep neural network for the detection of 3D objects, is shown in the below picture @fig-1.

![Overall structure of the SLS–Fusion neural network](images/2023-09-07_01-17.png){#fig-1}

- Firstly, an encoder–decoder based on  a ResNet network is designed to extract and fuse left/right features from stereo camera  images and project the LiDAR depth map.
- Secondly, the decoder network constructs a left and right depth map of optimized features through a depth cost volume model to predict  the corrected depth
- After the expected dense depth map is obtained, a pseudo-point cloud is generated using calibrated cameras.
- Finally, a LiDAR-based method for detecting 3D objects PointRCNN [@DBLP:conf/cvpr/ShiWL19] is applied to the predicted pseudo-point cloud.

## Characteristics of the Neural Network Architecture Used

- The main component of the SLS–Fusion neural network, used to fuse or separate LiDAR and stereo camera features (for an ablation study), is the encoder–decoder component  (see @fig-1 and @fig-2)

  ![SLS–Fusion encoder–decoder architecture:  The residual neural network blocks (ResNet  blocks) within the encoder are used to extract features from the LiDAR and stereo inputs. The fusion  process inside the decoder is accomplished through the use of addition and up-projection operators](images/2023-09-07_02-37.png){#fig-2}

- As shown in @fig-2, both the stereo camera and LiDAR encoders are composed  of a series of residual blocks from the neural network ResNet, followed by step-down  convolution to reduce the feature resolution of the input. ResNet is a group of residual  neural network blocks and each residual block is a stack of layers placed in such a way that  the output of one layer is taken and added to another deeper layer within the block, as shown in @fig-3  
  
  ![The structure of Stereo and LiDAR residual blocks inside the encoder/decoder of the SLS–  Fusion model. A stack of layers is grouped into blocks for stereo and LiDAR networks, conducted by  a step-down convolution direction and followed by a set of fusion blocks](images/2023-09-07_02-47.png){#fig-3}

- The main advantage of ResNet is its ability to prevent the accuracy from  saturating and degrading rapidly during the training of deeper neural networks (networks  with more than 20 layers). This advantage helps in choosing a network to be as deep as  needed for the problem at hand.
- The network of the decoder consists of adding the functions of both LiDAR and  stereo encoders, then up-projecting the result to progressively increase the resolution of  the features and generate a dense depth map as a decoder output.
- Because the sparse  input of LiDAR is heavily linked to the depth decoder output, features related to the  LiDAR sensor should contribute more to the decoder than features related to the stereo  sensor.